# TensorIOT Data Assignment 1

## Submitted by : Deepansh Singh
### Email - deepanshsingh8@gmail.com

Data Assignment 1

·         Download the data files from here - http://jmcauley.ucsd.edu/data/amazon/links.html

·         Apache spark tools locally and necessary tools

·         Download a review file with a million reviews.

·         Using Jupyter notebook create a program to read the million reviews and get the following.

·         Item having the least rating.

·         Item having most rating.

·         Item having the longest reviews.

·         Transform: change the date MM-DD-YYYY format.

·         Show a desired data frame operation which you learnt recently.

·         Convert the whole file into Parquet file after transforming.

·         Upload code to GitHub and complete Readme.md which anyone can understand.

·         Send GitHub link to HR.

### Method - 1 for reading Data

In [1]:
from pyspark.sql import SparkSession
import pandas as pd

# Initialize a Spark session
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('AmazonReview') \
    .getOrCreate()

# Specify the path to your JSON file
json_file_path = "Grocery_and_Gourmet_Food.json"

# Read the JSON file into a Spark DataFrame
spark_df = spark.read.json(json_file_path)

# Convert the Spark DataFrame to a Pandas DataFrame
pandas_df = spark_df.toPandas()

# Show the first few rows of the Pandas DataFrame
print(pandas_df.head())

         asin image  overall  \
0  4639725183  None      5.0   
1  4639725183  None      5.0   
2  4639725183  None      5.0   
3  4639725183  None      5.0   
4  4639725183  None      5.0   

                                          reviewText   reviewTime  \
0                                No adverse comment.  11 19, 2014   
1                          Gift for college student.  10 13, 2016   
2  If you like strong tea, this is for you. It mi...  11 21, 2015   
3  Love the tea. The flavor is way better than th...  08 12, 2015   
4  I have searched everywhere until I browsed Ama...  05 28, 2015   

       reviewerID     reviewerName style                        summary  \
0  A1QVBUH9E1V6I8   Jamshed Mathur  None                     Five Stars   
1  A3GEOILWLK86XM        itsjustme  None                 Great product.   
2  A32RD6L701BIGP  Krystal Clifton  None                         Strong   
3  A2UY1O1FBGKIE6          U. Kane  None                      Great tea   
4  A3QHVBQYDV7Z6U

### Method - 2 for reading data

In [2]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Grocery_and_Gourmet_Food.json.gz')

## Exploratory analysis

In [3]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"11 19, 2014",A1QVBUH9E1V6I8,4639725183,Jamshed Mathur,No adverse comment.,Five Stars,1416355200,NaN,NaN,NaN
1,5.0,True,"10 13, 2016",A3GEOILWLK86XM,4639725183,itsjustme,Gift for college student.,Great product.,1476316800,NaN,NaN,NaN
2,5.0,True,"11 21, 2015",A32RD6L701BIGP,4639725183,Krystal Clifton,"If you like strong tea, this is for you. It mi...",Strong,1448064000,NaN,NaN,NaN
3,5.0,True,"08 12, 2015",A2UY1O1FBGKIE6,4639725183,U. Kane,Love the tea. The flavor is way better than th...,Great tea,1439337600,NaN,NaN,NaN
4,5.0,True,"05 28, 2015",A3QHVBQYDV7Z6U,4639725183,The Nana,I have searched everywhere until I browsed Ama...,This is the tea I remembered!,1432771200,NaN,NaN,NaN


In [4]:
df.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'style', 'image'],
      dtype='object')

In [5]:
df.shape

(1143860, 12)

In [6]:
df.dtypes

overall           float64
verified             bool
reviewTime         object
reviewerID         object
asin               object
reviewerName       object
reviewText         object
summary            object
unixReviewTime      int64
vote               object
style              object
image              object
dtype: object

In [7]:
df.drop(['vote', 'style', 'image'], axis = 1, inplace = True)

In [8]:
df['asin'].value_counts()

asin
B00BUKL666    7387
B008QMX2SG    6228
B00D3M2QP4    6221
B00R7PWK7W    3387
B000X3TPHS    3030
              ... 
B000YAH13A       1
B000YINEYW       1
B00BB1DPOA       1
B0011MQSSY       1
B0012QDZBM       1
Name: count, Length: 41320, dtype: int64

#  Item having the least rating

In [9]:
Average_ratings = df.groupby(['asin'])['overall'].mean().reset_index()

In [10]:
least_rated_item = df.loc[df['overall'].idxmin()]
least_rated_item

overall                                                         1.0
verified                                                       True
reviewTime                                              03 11, 2016
reviewerID                                           A2PMV21JFDKJIU
asin                                                     4639725043
reviewerName                                            shopAholicr
reviewText        This tea looks like coffee grounds. Brewed it ...
summary                            does not look anything like tea.
unixReviewTime                                           1457654400
Name: 19, dtype: object

In [11]:
least_rated_items = Average_ratings.sort_values(['overall', 'asin'], ascending = [True, True]).rename(columns = {'asin':'ProductID', 'overall': 'AverageRating'})

In [12]:
# Top 5 least rated items
least_rated_items[:5]

,ProductID,AverageRating
5758,B000WW6VZO,1.0
6108,B000ZB3CRC,1.0
6741,B0014AJV82,1.0
7158,B001684NJO,1.0
3481,B000LKXQRS,1.2


#  Item having the Highest rating

In [13]:
most_rated_item = df.loc[df['overall'].idxmax()]
most_rated_item

overall                           5.0
verified                         True
reviewTime                11 19, 2014
reviewerID             A1QVBUH9E1V6I8
asin                       4639725183
reviewerName           Jamshed Mathur
reviewText        No adverse comment.
summary                    Five Stars
unixReviewTime             1416355200
Name: 0, dtype: object

In [14]:
#using the Average_ratings calculated above to display the names
highest_rated_items = Average_ratings.sort_values(['overall', 'asin'], ascending = [False, True]).rename(columns = {'asin':'ProductID', 'overall': 'AverageRating'})

In [15]:
# Top 5 highest rated items
highest_rated_items[:5]

,ProductID,AverageRating
64,B0000D17HA,5.0
69,B0000D8KJC,5.0
72,B0000D94PA,5.0
81,B0000D9N39,5.0
82,B0000D9N4P,5.0


# Item having longest Review

In [16]:
df['stringreview'] = df['reviewText'].astype(str)

In [17]:
df['stringreview'].astype(str)

0                                        No adverse comment.
1                                  Gift for college student.
2          If you like strong tea, this is for you. It mi...
3          Love the tea. The flavor is way better than th...
4          I have searched everywhere until I browsed Ama...
                                 ...                        
1143855    As a new vegan, it is sometimes difficult to r...
1143856    The best thing ever is ordering a product you ...
1143857    I used to love ranch before I became vegan. It...
1143858    I cannot have dairy nor gluten.  This is as cl...
1143859    Needs improvement to make it taste like real r...
Name: stringreview, Length: 1143860, dtype: object

In [18]:
index_longest_review = df['reviewText'].astype(str).apply(len).idxmax()

In [19]:
#Item with details having longest review
item_with_longest_review = df.loc[index_longest_review]
item_with_longest_review

overall                                                         2.0
verified                                                       True
reviewTime                                              10 15, 2009
reviewerID                                            AJNJNA093JUW0
asin                                                     B001DEX99Q
reviewerName                                           Serendipity.
reviewText        I am a sweet-aholic. That's my main downfall w...
summary           The Real Review of Vitalicious Vitatops (Brown...
unixReviewTime                                           1255564800
stringreview      I am a sweet-aholic. That's my main downfall w...
Name: 298949, dtype: object

In [20]:
length = len(item_with_longest_review['stringreview'])
#Length of the review
length

19057

# Converting reviewTime to date MM-DD-YYYY format.

In [21]:
df['reviewDate'] = pd.to_datetime(df['unixReviewTime'], unit='s').dt.strftime('%m-%d-%Y')

In [22]:
df.drop(['reviewTime', 'unixReviewTime', 'reviewText'], axis = 1, inplace = True)

In [23]:
df = df.rename(columns = {'stringreview':'Review'})

 # Show a desired data frame operation which you learnt recently.

In [24]:
print(' '.join(df['Review'].tolist()[:2000]))

No adverse comment. Gift for college student. If you like strong tea, this is for you. It might even be a little too strong for me. Love the tea. The flavor is way better than the regular Lipton black tea. Definetly worth the money. I have searched everywhere until I browsed Amazon and found it!  This is the tea Lipton should be selling on grocery store shelves.  The stuff I last purchased is just awful, and nowhere near as good as I remember. Tea made with Lipton Yellow Label teabags is much better than tea made with regular Lipton teabags. I love this tea!  Okay, I'm not a high falutin' tea snob, but I first enjoyed this Lipton yellow label tea when on a trip to Singapore, and whether it was the room service, the company, or the country - it left such a favorable impression on me, I search it out whenever possible.  Maybe it's the same as the USA's white label Lipton tea, but the yellow label tea appears to me to be smoother and more mellow.  I would highly recommend it - give it a t

In [25]:
!pip install contractions
import contractions
def cont_to_exp(x):
    if type(x) is str:
        x = x.replace('\\','')
        x = x.replace('...','')
        x = x.replace(' - ', '')
        x = x.replace('    ', '')
        x = contractions.fix(x)
        return x
    else:
        return x

Defaulting to user installation because normal site-packages is not writeable


In [26]:
df['Review'] = df['Review'].apply(lambda x: cont_to_exp(x))

In [27]:
print(' '.join(df['Review'].tolist()[:1000]))

No adverse comment. Gift for college student. If you like strong tea, this is for you. It might even be a little too strong for me. Love the tea. The flavor is way better than the regular Lipton black tea. Definetly worth the money. I have searched everywhere until I browsed Amazon and found it!  This is the tea Lipton should be selling on grocery store shelves.  The stuff I last purchased is just awful, and nowhere near as good as I remember. Tea made with Lipton Yellow Label teabags is much better than tea made with regular Lipton teabags. I love this tea!  Okay, I am not a high falutin' tea snob, but I first enjoyed this Lipton yellow label tea when on a trip to Singapore, and whether it was the room service, the company, or the countryit left such a favorable impression on me, I search it out whenever possible.  Maybe it is the same as the USA's white label Lipton tea, but the yellow label tea appears to me to be smoother and more mellow.  I would highly recommend itgive it a try! 

In [ ]:
# Feature Engineering
!pip install TextBlob
from textblob import TextBlob

## Uses prebuilt library to calculate polarity of sentiment
df['polarity'] = df['Review'].apply(lambda x: TextBlob(x).sentiment.polarity)
df.head()

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
## Gives the number of characters in the narrative
df['Character_len'] = df['Review'].apply(lambda x: len(x))

## Gives the length of the narrative - total words
df['Narrative_len'] = df['Review'].apply(lambda x: len(x.split()))

In [ ]:
df.head()

In [ ]:
##To avoid spaces in the text, we calculate the word length this way
def avg_word_len(x):
    words = x.split()
    if len(words) != 0:
        word_len = 0
        for word in words:
            word_len = word_len + len(word)
        return word_len/len(words)

In [ ]:
df['avg_word_len'] = df['Review'].apply(lambda x: avg_word_len(x))
df.head()

# Distribution of Sentiment Polarity

In [ ]:
!pip install cufflinks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import cufflinks as cf
from plotly.offline import iplot
%matplotlib inline

In [ ]:
py.offline.init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
df['polarity'].iplot(kind = 'hist', colors = 'red', bins = 20,title = 'Sentiment Polarity Distribution')

# Distribution of Narrative Length and Character Length

In [ ]:
df['Character_len'].iplot(kind = "hist",bins = 50, colors = 'red', xTitle = "Character Length in Narrative", yTitle = "Count", title = "Narrative Text Character Distribution")

In [ ]:
df['Narrative_len'].iplot(kind = "hist",bins = 50, colors = 'red', xTitle = "Narrative Length", yTitle = "Count", title = "Narrative Length Distribution")

In [ ]:
df['avg_word_len'].iplot(kind = "hist", colors = 'red', xTitle = "Avg Word Length", bins = 50, yTitle = "Count", title = "Narrative Text Length Avg Word Distribution")

# Saving to Parquet file
## Further Bigram and Trigram analysis - but it is out of scope for this assignment

In [ ]:
df.to_parquet('Groceries_and_Gourmet_Food.parquet', index=False)

In [ ]:
df